# Latin Square Komutatif atas Aljabar Max-Plus

Program ini mencari semua pasangan Latin square A dan B (dengan A ≠ B) yang komutatif di bawah perkalian max-plus, yaitu: **A ⊗ B = B ⊗ A**

Dengan entri: {1, 2, 3, 4}

In [99]:
import numpy as np
from itertools import permutations, product
import warnings
warnings.filterwarnings('ignore')

def is_latin_square(matrix):
    """Cek apakah matriks adalah Latin square"""
    n = len(matrix)
    # Cek setiap baris dan kolom memiliki elemen unik
    for i in range(n):
        if len(set(matrix[i])) != n:  # Baris
            return False
        if len(set(matrix[:, i])) != n:  # Kolom
            return False
    return True

def maxplus_mult(A, B):
    """Perkalian max-plus: (A ⊗ B)_ij = max_k(A_ik + B_kj)"""
    n = A.shape[0]
    C = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            C[i, j] = max(A[i, k] + B[k, j] for k in range(n))
    return C

def matrices_equal(A, B):
    """Cek apakah dua matriks sama"""
    return np.allclose(A, B)

def is_circulant(matrix):
    """Cek apakah matriks adalah circulant matrix
    Circulant matrix: setiap baris adalah cyclic shift ke KIRI dari baris sebelumnya"""
    n = len(matrix)
    first_row = matrix[0]
    
    for i in range(1, n):
        # Baris ke-i harus sama dengan cyclic shift baris pertama sebanyak i posisi ke KIRI
        # Shift ke kiri = roll dengan nilai negatif
        expected_row = np.roll(first_row, -i)
        if not np.array_equal(matrix[i], expected_row):
            return False
    return True

def generate_all_latin_squares(n, symbols):
    """Generate semua Latin square n×n dengan simbol tertentu"""
    latin_squares = []
    
    # Generate semua permutasi baris pertama
    for first_row in permutations(symbols):
        # Coba semua kemungkinan baris berikutnya
        def backtrack(matrix, row):
            if row == n:
                mat = np.array(matrix)
                if is_latin_square(mat):
                    latin_squares.append(mat.copy())
                return
            
            for perm in permutations(symbols):
                matrix.append(list(perm))
                # Quick check: cek kolom sejauh ini
                valid = True
                for col in range(n):
                    col_vals = [matrix[r][col] for r in range(row + 1)]
                    if len(col_vals) != len(set(col_vals)):
                        valid = False
                        break
                
                if valid:
                    backtrack(matrix, row + 1)
                matrix.pop()
        
        backtrack([list(first_row)], 1)
    
    return latin_squares

print("Generating semua Latin square 4×4 dengan entri {1,2,3,4}...")
print("(Proses ini mungkin butuh beberapa detik...)")

Generating semua Latin square 4×4 dengan entri {1,2,3,4}...
(Proses ini mungkin butuh beberapa detik...)


In [87]:
# Generate semua Latin square
symbols = [1, 2, 3, 4]
all_latin_squares = generate_all_latin_squares(4, symbols)

print(f"\nTotal Latin square yang ditemukan: {len(all_latin_squares)}")
print(f"\nContoh Latin square pertama:")
print(all_latin_squares[0])


Total Latin square yang ditemukan: 576

Contoh Latin square pertama:
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]


In [88]:
# Cari semua pasangan Latin square yang komutatif
print("\n" + "="*80)
print("MENCARI PASANGAN LATIN SQUARE KOMUTATIF (A ⊗ B = B ⊗ A)")
print("="*80)

commutative_pairs = []
n_squares = len(all_latin_squares)

# Cek semua pasangan
for i in range(n_squares):
    for j in range(i+1, n_squares):  # j > i agar tidak duplikat
        A = all_latin_squares[i]
        B = all_latin_squares[j]
        
        # Hitung A ⊗ B dan B ⊗ A
        A_mult_B = maxplus_mult(A, B)
        B_mult_A = maxplus_mult(B, A)
        
        # Cek komutativitas
        if matrices_equal(A_mult_B, B_mult_A):
            commutative_pairs.append({
                'A': A,
                'B': B,
                'A_index': i,
                'B_index': j,
                'product': A_mult_B
            })

print(f"\nJumlah pasangan komutatif yang ditemukan: {len(commutative_pairs)}")
print(f"Total pasangan yang dicek: {n_squares * (n_squares - 1) // 2}")


MENCARI PASANGAN LATIN SQUARE KOMUTATIF (A ⊗ B = B ⊗ A)

Jumlah pasangan komutatif yang ditemukan: 4752
Total pasangan yang dicek: 165600


In [89]:
# Tampilkan SEMUA pasangan komutatif
print("\n" + "="*80)
print("SEMUA PASANGAN LATIN SQUARE KOMUTATIF")
print("="*80)

for idx, pair in enumerate(commutative_pairs, 1):
    print(f"\n{'='*80}")
    print(f"PASANGAN #{idx}")
    print(f"{'='*80}")
    
    A = pair['A']
    B = pair['B']
    product = pair['product']
    
    print(f"\nMatriks A (Latin Square #{pair['A_index'] + 1}):")
    print(A.astype(int))
    
    print(f"\nMatriks B (Latin Square #{pair['B_index'] + 1}):")
    print(B.astype(int))
    
    print(f"\nA ⊗ B = B ⊗ A (Hasil Perkalian Max-Plus):")
    print(product.astype(int))
    
    # Verifikasi komutativitas
    A_mult_B = maxplus_mult(A, B)
    B_mult_A = maxplus_mult(B, A)
    print(f"\n✓ Komutatif: A ⊗ B = B ⊗ A? {matrices_equal(A_mult_B, B_mult_A)}")
    
    # Analisis tambahan
    max_entry = np.max(product)
    print(f"  - Entri terbesar di hasil perkalian: {int(max_entry)}")
    print(f"  - Sum entri diagonal hasil: {int(np.trace(product))}")

print(f"\n{'='*80}")
print(f"RINGKASAN: Ditemukan {len(commutative_pairs)} pasangan Latin square komutatif")
print(f"{'='*80}")


SEMUA PASANGAN LATIN SQUARE KOMUTATIF

PASANGAN #1

Matriks A (Latin Square #1):
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B (Latin Square #27):
[[1 2 4 3]
 [2 1 3 4]
 [4 3 1 2]
 [3 4 2 1]]

A ⊗ B = B ⊗ A (Hasil Perkalian Max-Plus):
[[7 8 6 6]
 [8 7 6 6]
 [6 6 7 8]
 [6 6 8 7]]

✓ Komutatif: A ⊗ B = B ⊗ A? True
  - Entri terbesar di hasil perkalian: 8
  - Sum entri diagonal hasil: 28

PASANGAN #2

Matriks A (Latin Square #1):
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B (Latin Square #30):
[[1 2 4 3]
 [2 3 1 4]
 [4 1 3 2]
 [3 4 2 1]]

A ⊗ B = B ⊗ A (Hasil Perkalian Max-Plus):
[[7 8 6 6]
 [8 7 7 6]
 [6 7 7 8]
 [6 6 8 7]]

✓ Komutatif: A ⊗ B = B ⊗ A? True
  - Entri terbesar di hasil perkalian: 8
  - Sum entri diagonal hasil: 28

PASANGAN #3

Matriks A (Latin Square #1):
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B (Latin Square #57):
[[1 3 2 4]
 [3 1 4 2]
 [2 4 1 3]
 [4 2 3 1]]

A ⊗ B = B ⊗ A (Hasil Perkalian Max-Plus):
[[8 7 7 6]
 [7 8 6 7]
 [7 6 8 7]
 

## Analisis Dekomposisi Matriks Permutasi

Analisis nilai terbesar dari entri hasil perkalian (untuk melihat koefisien 8, 7, 6, 5)

In [90]:
# Analisis distribusi nilai maksimum dalam hasil perkalian
print("\n" + "="*80)
print("ANALISIS NILAI MAKSIMUM DALAM HASIL PERKALIAN")
print("="*80)

max_values_distribution = {}
for pair in commutative_pairs:
    max_val = int(np.max(pair['product']))
    if max_val not in max_values_distribution:
        max_values_distribution[max_val] = 0
    max_values_distribution[max_val] += 1

print("\nDistribusi nilai maksimum:")
for val in sorted(max_values_distribution.keys(), reverse=True):
    count = max_values_distribution[val]
    print(f"  Nilai max = {val}: {count} pasangan")

# Tampilkan contoh untuk setiap kategori
print("\n" + "="*80)
print("CONTOH UNTUK SETIAP KATEGORI NILAI MAKSIMUM")
print("="*80)

shown_categories = set()
for idx, pair in enumerate(commutative_pairs, 1):
    max_val = int(np.max(pair['product']))
    
    if max_val not in shown_categories:
        shown_categories.add(max_val)
        print(f"\n--- CONTOH: Nilai Maksimum = {max_val} ---")
        print(f"Pasangan #{idx}")
        print(f"\nMatriks A:")
        print(pair['A'].astype(int))
        print(f"\nMatriks B:")
        print(pair['B'].astype(int))
        print(f"\nA ⊗ B:")
        print(pair['product'].astype(int))


ANALISIS NILAI MAKSIMUM DALAM HASIL PERKALIAN

Distribusi nilai maksimum:
  Nilai max = 8: 4752 pasangan

CONTOH UNTUK SETIAP KATEGORI NILAI MAKSIMUM

--- CONTOH: Nilai Maksimum = 8 ---
Pasangan #1

Matriks A:
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B:
[[1 2 4 3]
 [2 1 3 4]
 [4 3 1 2]
 [3 4 2 1]]

A ⊗ B:
[[7 8 6 6]
 [8 7 6 6]
 [6 6 7 8]
 [6 6 8 7]]


## Export Hasil ke File

Untuk mempermudah analisis lebih lanjut

In [91]:
# Export ke file teks
output_file = "hasil_latin_square_komutatif.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("SEMUA PASANGAN LATIN SQUARE KOMUTATIF ATAS ALJABAR MAX-PLUS\n")
    f.write("Entri: {1, 2, 3, 4}\n")
    f.write("="*80 + "\n\n")
    f.write(f"Total pasangan ditemukan: {len(commutative_pairs)}\n\n")
    
    for idx, pair in enumerate(commutative_pairs, 1):
        f.write(f"\n{'='*80}\n")
        f.write(f"PASANGAN #{idx}\n")
        f.write(f"{'='*80}\n")
        
        A = pair['A']
        B = pair['B']
        product = pair['product']
        
        f.write(f"\nMatriks A (Latin Square #{pair['A_index'] + 1}):\n")
        for row in A.astype(int):
            f.write("  " + str(row) + "\n")
        
        f.write(f"\nMatriks B (Latin Square #{pair['B_index'] + 1}):\n")
        for row in B.astype(int):
            f.write("  " + str(row) + "\n")
        
        f.write(f"\nA ⊗ B = B ⊗ A (Hasil Perkalian Max-Plus):\n")
        for row in product.astype(int):
            f.write("  " + str(row) + "\n")
        
        f.write(f"\nNilai maksimum: {int(np.max(product))}\n")
        f.write(f"Trace (sum diagonal): {int(np.trace(product))}\n")

print(f"✓ Hasil berhasil disimpan ke file: {output_file}")
print(f"  Total: {len(commutative_pairs)} pasangan komutatif")
print(f"  Semua pasangan memiliki nilai maksimum = 8 dalam hasil perkalian")

✓ Hasil berhasil disimpan ke file: hasil_latin_square_komutatif.txt
  Total: 4752 pasangan komutatif
  Semua pasangan memiliki nilai maksimum = 8 dalam hasil perkalian


## Analisis Matriks Permutasi

Dekomposisi Latin square menjadi matriks permutasi dan cek komutativitas pada koefisien terbesar

In [92]:
def extract_permutation_matrices(latin_square):
    """
    Ekstrak matriks permutasi dari Latin square.
    Untuk setiap nilai v, buat matriks permutasi P_v dimana P_v[i,j] = 1 jika L[i,j] = v
    
    Returns: dict {nilai: matriks_permutasi}
    """
    n = latin_square.shape[0]
    unique_vals = sorted(np.unique(latin_square))
    
    perm_matrices = {}
    for val in unique_vals:
        P = np.zeros((n, n), dtype=int)
        for i in range(n):
            for j in range(n):
                if latin_square[i, j] == val:
                    P[i, j] = 1
        perm_matrices[int(val)] = P
    
    return perm_matrices

def normal_matrix_mult(A, B):
    """Perkalian matriks biasa (bukan max-plus)"""
    return np.matmul(A, B)

def get_contributing_permutations(A, B, A_perm, B_perm, target_sum):
    """
    Cari pasangan matriks permutasi yang berkontribusi pada nilai target_sum
    dalam hasil perkalian max-plus.
    
    Returns: list of tuples (coef_a, coef_b, P_a, P_b) dimana coef_a + coef_b = target_sum
    """
    contributors = []
    
    for coef_a in A_perm.keys():
        for coef_b in B_perm.keys():
            if coef_a + coef_b == target_sum:
                P_a = A_perm[coef_a]
                P_b = B_perm[coef_b]
                contributors.append((coef_a, coef_b, P_a, P_b))
    
    return contributors

print("Fungsi untuk analisis matriks permutasi sudah siap!")

Fungsi untuk analisis matriks permutasi sudah siap!


In [100]:
# Analisis semua pasangan komutatif: cari yang matriks permutasinya TIDAK komutatif
print("\n" + "="*80)
print("MENCARI PASANGAN DENGAN MATRIKS PERMUTASI TIDAK KOMUTATIF")
print("="*80)
print("\nKriteria:")
print("  ✓ A ⊗ B = B ⊗ A (Latin square komutatif)")
print("  ✗ P_k(A) ≠ P_k(B) (matriks permutasi BERBEDA)")
print("  ✗ P_k(A) × P_k(B) ≠ P_k(B) × P_k(A) (tidak komutatif)")
print("  ✗ Setidaknya salah satu dari A atau B bukan circulant")
print("  ✗ HANYA analisis koefisien 6 (dari 3+3)")
print()

non_commutative_perm_pairs = []

for idx, pair in enumerate(commutative_pairs):
    A = pair['A']
    B = pair['B']
    
    # FILTER: Skip jika A atau B salah satu circulant
    if is_circulant(A) or is_circulant(B):
        continue
    
    # Ekstrak matriks permutasi
    A_perm = extract_permutation_matrices(A)
    B_perm = extract_permutation_matrices(B)
    
    # Cek untuk setiap koefisien
    found_non_commutative = False
    non_comm_details = []
    
    # HANYA koefisien 3 (max-plus 6)
    for coef in [3]:
        P_a = A_perm[coef]
        P_b = B_perm[coef]
        
        # SKIP jika matriks permutasinya SAMA (pasti komutatif)
        if np.allclose(P_a, P_b):
            continue
        
        # Hitung P_a × P_b dan P_b × P_a (perkalian matriks biasa)
        P_ab = normal_matrix_mult(P_a, P_b)
        P_ba = normal_matrix_mult(P_b, P_a)
        
        # Cek apakah komutatif
        if not np.allclose(P_ab, P_ba):
            found_non_commutative = True
            non_comm_details.append({
                'target': coef,
                'coef_a': coef,
                'coef_b': coef,
                'P_a': P_a,
                'P_b': P_b,
                'P_ab': P_ab,
                'P_ba': P_ba
            })
    
    if found_non_commutative:
        pair['non_comm_details'] = non_comm_details
        non_commutative_perm_pairs.append(pair)

print(f"\n✓ Ditemukan {len(non_commutative_perm_pairs)} pasangan!")
print(f"  Dari total {len(commutative_pairs)} pasangan komutatif")


MENCARI PASANGAN DENGAN MATRIKS PERMUTASI TIDAK KOMUTATIF

Kriteria:
  ✓ A ⊗ B = B ⊗ A (Latin square komutatif)
  ✗ P_k(A) ≠ P_k(B) (matriks permutasi BERBEDA)
  ✗ P_k(A) × P_k(B) ≠ P_k(B) × P_k(A) (tidak komutatif)
  ✗ Setidaknya salah satu dari A atau B bukan circulant
  ✗ HANYA analisis koefisien 6 (dari 3+3)


✓ Ditemukan 704 pasangan!
  Dari total 4752 pasangan komutatif


In [94]:
# Tampilkan contoh-contoh pasangan dengan matriks permutasi tidak komutatif
print("\n" + "="*80)
print("CONTOH PASANGAN: Latin Square Komutatif, Matriks Permutasi TIDAK Komutatif")
print("="*80)

# Tampilkan 5 contoh pertama (atau semua jika kurang dari 5)
num_examples = min(5, len(non_commutative_perm_pairs))

for i in range(num_examples):
    pair = non_commutative_perm_pairs[i]
    
    print(f"\n{'='*80}")
    print(f"CONTOH #{i+1}")
    print(f"{'='*80}")
    
    A = pair['A']
    B = pair['B']
    product = pair['product']
    
    print(f"\nMatriks A:")
    print(A.astype(int))
    
    print(f"\nMatriks B:")
    print(B.astype(int))
    
    print(f"\nA ⊗ B = B ⊗ A (Max-Plus):")
    print(product.astype(int))
    print("✓ KOMUTATIF dalam max-plus")
    
    # Tampilkan detail matriks permutasi yang tidak komutatif
    print(f"\n--- Matriks Permutasi yang TIDAK Komutatif ---")
    
    detail = pair['non_comm_details'][0]  # Ambil contoh pertama
    
    print(f"\nUntuk koefisien total = {detail['target']} (dari {detail['coef_a']} + {detail['coef_b']}):")
    
    print(f"\nMatriks Permutasi P_{detail['coef_a']} dari A:")
    print(detail['P_a'])
    
    print(f"\nMatriks Permutasi P_{detail['coef_b']} dari B:")
    print(detail['P_b'])
    
    print(f"\nP_{detail['coef_a']} × P_{detail['coef_b']}:")
    print(detail['P_ab'].astype(int))
    
    print(f"\nP_{detail['coef_b']} × P_{detail['coef_a']}:")
    print(detail['P_ba'].astype(int))
    
    print(f"\n✗ TIDAK KOMUTATIF: P_{detail['coef_a']} × P_{detail['coef_b']} ≠ P_{detail['coef_b']} × P_{detail['coef_a']}")

print(f"\n{'='*80}")
print(f"Total: {len(non_commutative_perm_pairs)} pasangan ditemukan")
print(f"{'='*80}")


CONTOH PASANGAN: Latin Square Komutatif, Matriks Permutasi TIDAK Komutatif

CONTOH #1

Matriks A:
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B:
[[1 2 4 3]
 [2 3 1 4]
 [4 1 3 2]
 [3 4 2 1]]

A ⊗ B = B ⊗ A (Max-Plus):
[[7 8 6 6]
 [8 7 7 6]
 [6 7 7 8]
 [6 6 8 7]]
✓ KOMUTATIF dalam max-plus

--- Matriks Permutasi yang TIDAK Komutatif ---

Untuk koefisien total = 3 (dari 3 + 3):

Matriks Permutasi P_3 dari A:
[[0 0 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]]

Matriks Permutasi P_3 dari B:
[[0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]]

P_3 × P_3:
[[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]]

P_3 × P_3:
[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]

✗ TIDAK KOMUTATIF: P_3 × P_3 ≠ P_3 × P_3

CONTOH #2

Matriks A:
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B:
[[1 4 2 3]
 [4 3 1 2]
 [2 1 3 4]
 [3 2 4 1]]

A ⊗ B = B ⊗ A (Max-Plus):
[[7 6 8 7]
 [6 6 7 8]
 [8 7 6 6]
 [7 8 6 7]]
✓ KOMUTATIF dalam max-plus

--- Matriks Permutasi yang TIDAK Komutatif ---

Untuk koefisien total = 3 (dar

In [102]:
# Statistik detail: pada koefisien berapa yang paling sering tidak komutatif?
print("\n" + "="*80)
print("STATISTIK: Koefisien Relevan (8, 7, 6, 5)")
print("="*80)

# Hanya fokus ke koefisien yang relevan (besar)
# Koefisien 8 (4+4) - sudah pasti 0
# Koefisien 6 (3+3) - dari analisis sebelumnya
coef_maxplus_stats = {8: 0, 6: 0}

for pair in non_commutative_perm_pairs:
    checked_targets = set()
    for detail in pair['non_comm_details']:
        target = detail['target']
        # Hanya ambil yang relevan (koefisien 6 dari 3+3)
        if target == 3:  # 3+3 = 6
            maxplus_coef = 6
            if maxplus_coef not in checked_targets:
                coef_maxplus_stats[maxplus_coef] += 1
                checked_targets.add(maxplus_coef)

print(f"\nKoefisien yang sama (a+a):")
print(f"  - Koefisien 8 (dari 4+4): {coef_maxplus_stats[8]} pasangan [SELALU KOMUTATIF]")
print(f"  - Koefisien 6 (dari 3+3): {coef_maxplus_stats[6]} pasangan")

print(f"\nKoefisien campuran (a+b, a≠b):")
print(f"  - Koefisien 7 (dari 4+3): {mixed_stats[7]} pasangan")
print(f"  - Koefisien 5 (dari 4+1, 3+2): {mixed_stats[5]} pasangan")

print(f"\nTotal pasangan dengan matriks permutasi tidak komutatif: {len(non_commutative_perm_pairs)}")
print(f"Persentase: {len(non_commutative_perm_pairs)/len(commutative_pairs)*100:.1f}%")


STATISTIK: Koefisien Relevan (8, 7, 6, 5)

Koefisien yang sama (a+a):
  - Koefisien 8 (dari 4+4): 0 pasangan [SELALU KOMUTATIF]
  - Koefisien 6 (dari 3+3): 704 pasangan

Koefisien campuran (a+b, a≠b):
  - Koefisien 7 (dari 4+3): 224 pasangan
  - Koefisien 5 (dari 4+1, 3+2): 2364 pasangan

Total pasangan dengan matriks permutasi tidak komutatif: 704
Persentase: 14.8%


In [103]:
# Export hasil ke file - GABUNGKAN SEMUA (koefisien sama + campuran)
output_file2 = "hasil_non_commutative_permutation.txt"

# Gabungkan semua pasangan unik
all_pairs_dict = {}
for pair in non_commutative_perm_pairs:
    key = (pair['A_index'], pair['B_index'])
    all_pairs_dict[key] = pair
for pair in mixed_non_commutative_pairs:
    key = (pair['A_index'], pair['B_index'])
    if key not in all_pairs_dict:
        all_pairs_dict[key] = pair
    else:
        # Gabungkan detail
        if 'mixed_non_comm_details' in pair:
            all_pairs_dict[key]['mixed_non_comm_details'] = pair['mixed_non_comm_details']

all_pairs = list(all_pairs_dict.values())

with open(output_file2, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("PASANGAN LATIN SQUARE KOMUTATIF\n")
    f.write("DENGAN MATRIKS PERMUTASI TIDAK KOMUTATIF\n")
    f.write("="*80 + "\n\n")
    f.write("Temuan Penting untuk TA:\n")
    f.write("Latin square A dan B yang komutatif (A ⊗ B = B ⊗ A)\n")
    f.write("TAPI matriks permutasi tidak komutatif untuk koefisien tertentu!\n\n")
    f.write(f"Total pasangan unik ditemukan: {len(all_pairs)}\n")
    f.write(f"Dari {len(commutative_pairs)} pasangan komutatif total\n\n")
    
    # Statistik circulant
    both_circ = sum(1 for p in all_pairs if is_circulant(p['A']) and is_circulant(p['B']))
    a_only_circ = sum(1 for p in all_pairs if is_circulant(p['A']) and not is_circulant(p['B']))
    b_only_circ = sum(1 for p in all_pairs if not is_circulant(p['A']) and is_circulant(p['B']))
    neither_circ = sum(1 for p in all_pairs if not is_circulant(p['A']) and not is_circulant(p['B']))
    
    f.write("="*80 + "\n")
    f.write("FILTER CIRCULANT\n")
    f.write("="*80 + "\n\n")
    f.write("Distribusi pasangan berdasarkan circulant:\n")
    f.write(f"  - Kedua A dan B circulant: {both_circ} pasangan (DIFILTER!)\n")
    f.write(f"  - Hanya A circulant: {a_only_circ} pasangan\n")
    f.write(f"  - Hanya B circulant: {b_only_circ} pasangan\n")
    f.write(f"  - Tidak ada yang circulant: {neither_circ} pasangan\n\n")
    
    # Statistik
    f.write("="*80 + "\n")
    f.write("STATISTIK (Koefisien Relevan: 8, 7, 6, 5)\n")
    f.write("="*80 + "\n\n")
    f.write("Koefisien yang sama (a+a):\n")
    f.write(f"  - Koefisien 8 (dari 4+4): {coef_maxplus_stats[8]} pasangan [SELALU KOMUTATIF]\n")
    f.write(f"  - Koefisien 6 (dari 3+3): {coef_maxplus_stats[6]} pasangan\n\n")
    f.write("Koefisien campuran (a+b, a≠b):\n")
    f.write(f"  - Koefisien 7 (dari 4+3): {mixed_stats[7]} pasangan\n")
    f.write(f"  - Koefisien 5 (dari 4+1, 3+2): {mixed_stats[5]} pasangan\n\n")
    
    # Detail semua pasangan
    for idx, pair in enumerate(all_pairs, 1):
        f.write(f"\n{'='*80}\n")
        f.write(f"PASANGAN #{idx}\n")
        f.write(f"{'='*80}\n")
        
        A = pair['A']
        B = pair['B']
        product = pair['product']
        
        a_circ = is_circulant(A)
        b_circ = is_circulant(B)
        
        f.write(f"\nMatriks A (circulant: {a_circ}):\n")
        for row in A.astype(int):
            f.write("  " + str(row) + "\n")
        
        f.write(f"\nMatriks B (circulant: {b_circ}):\n")
        for row in B.astype(int):
            f.write("  " + str(row) + "\n")
        
        f.write(f"\nA ⊗ B = B ⊗ A (KOMUTATIF):\n")
        for row in product.astype(int):
            f.write("  " + str(row) + "\n")
        
        # Detail matriks permutasi tidak komutatif
        f.write(f"\n--- Matriks Permutasi TIDAK Komutatif ---\n")
        
        # Koefisien sama (6)
        if 'non_comm_details' in pair:
            for detail in pair['non_comm_details'][:1]:
                k = detail['target']
                maxplus_k = k * 2
                f.write(f"\nKoefisien {maxplus_k} (dari {k}+{k}):\n")
                f.write(f"P_{k}(A) × P_{k}(B) ≠ P_{k}(B) × P_{k}(A)\n")
        
        # Koefisien campuran (7, 5)
        if 'mixed_non_comm_details' in pair:
            for detail in pair['mixed_non_comm_details'][:1]:
                target = detail['target']
                a = detail['coef_a']
                b = detail['coef_b']
                f.write(f"\nKoefisien {target} (dari {a}+{b}):\n")
                f.write(f"P_{a}(A) × P_{b}(B) ≠ P_{b}(B) × P_{a}(A)\n")

print(f"\n✓ Hasil lengkap disimpan ke: {output_file2}")
print(f"  Total: {len(all_pairs)} pasangan UNIK dengan matriks permutasi tidak komutatif")
print(f"  (Gabungan koefisien sama + campuran)")
print(f"\n🎯 KESIMPULAN PENTING:")
print(f"  {len(all_pairs)}/{len(commutative_pairs)} = {len(all_pairs)/len(commutative_pairs)*100:.1f}%")
print(f"  dari Latin square komutatif memiliki matriks permutasi TIDAK komutatif!")
print(f"\n  Fokus ke koefisien RELEVAN (8, 7, 6, 5):")
print(f"  ✓ Koefisien 8 (4+4): SELALU komutatif!")
print(f"  ✗ Koefisien 7 (4+3): {mixed_stats[7]} pasangan tidak komutatif")
print(f"  ✗ Koefisien 6 (3+3): {coef_maxplus_stats[6]} pasangan tidak komutatif")
print(f"  ✗ Koefisien 5: {mixed_stats[5]} pasangan tidak komutatif")


✓ Hasil lengkap disimpan ke: hasil_non_commutative_permutation.txt
  Total: 2556 pasangan UNIK dengan matriks permutasi tidak komutatif
  (Gabungan koefisien sama + campuran)

🎯 KESIMPULAN PENTING:
  2556/4752 = 53.8%
  dari Latin square komutatif memiliki matriks permutasi TIDAK komutatif!

  Fokus ke koefisien RELEVAN (8, 7, 6, 5):
  ✓ Koefisien 8 (4+4): SELALU komutatif!
  ✗ Koefisien 7 (4+3): 224 pasangan tidak komutatif
  ✗ Koefisien 6 (3+3): 704 pasangan tidak komutatif
  ✗ Koefisien 5: 2364 pasangan tidak komutatif


In [101]:
# Analisis TAMBAHAN: Cek koefisien campuran (7, 5)  
# FILTER KETAT: P_a(A) HARUS BEDA dari P_b(A) DAN P_a(B) HARUS BEDA dari P_b(B)
# FILTER CIRCULANT: Setidaknya salah satu dari A atau B bukan circulant
print("\n" + "="*80)
print("ANALISIS TAMBAHAN: Koefisien Campuran (7 dan 5)")
print("="*80)
print("\nCek: P_a(A) × P_b(B) vs P_b(B) × P_a(A) dimana a ≠ b")
print("  - Koefisien 7: dari 4+3 atau 3+4")
print("  - Koefisien 5: dari 4+1, 3+2, 2+3, atau 1+4")
print("  FILTER: P_a(A) ≠ P_b(A) DAN P_a(B) ≠ P_b(B) DAN P_a(A) ≠ P_a(B) DAN P_b(A) ≠ P_b(B)")
print("  FILTER CIRCULANT: Setidaknya salah satu dari A atau B bukan circulant")
print()

mixed_non_commutative_pairs = []

for idx, pair in enumerate(commutative_pairs):
    A = pair['A']
    B = pair['B']
    
    # FILTER: Skip jika A atau B salah satu circulant
    if is_circulant(A) or is_circulant(B):
        continue
    
    # Ekstrak matriks permutasi
    A_perm = extract_permutation_matrices(A)
    B_perm = extract_permutation_matrices(B)
    
    found_mixed_non_commutative = False
    mixed_non_comm_details = []
    
    # Cek semua kombinasi a + b = target untuk target 7 dan 5
    for target in [7, 5]:
        for a in [4, 3, 2, 1]:
            for b in [4, 3, 2, 1]:
                if a + b == target and a != b:  # Pastikan campuran (a ≠ b)
                    P_a_A = A_perm[a]
                    P_b_B = B_perm[b]
                    P_a_B = B_perm[a]
                    P_b_A = A_perm[b]
                    
                    # FILTER KETAT: Semua harus BEDA
                    # 1. P_a(A) ≠ P_b(A)
                    if np.allclose(P_a_A, P_b_A):
                        continue
                    # 2. P_a(B) ≠ P_b(B)
                    if np.allclose(P_a_B, P_b_B):
                        continue
                    # 3. P_a(A) ≠ P_a(B) (matriks permutasi untuk koef a berbeda di A dan B)
                    if np.allclose(P_a_A, P_a_B):
                        continue
                    # 4. P_b(A) ≠ P_b(B) (matriks permutasi untuk koef b berbeda di A dan B)
                    if np.allclose(P_b_A, P_b_B):
                        continue
                    
                    # Cek P_a(A) × P_b(B) vs P_b(B) × P_a(A)
                    P_ab = normal_matrix_mult(P_a_A, P_b_B)
                    P_ba_swapped = normal_matrix_mult(P_b_B, P_a_A)
                    
                    if not np.allclose(P_ab, P_ba_swapped):
                        found_mixed_non_commutative = True
                        mixed_non_comm_details.append({
                            'target': target,
                            'coef_a': a,
                            'coef_b': b,
                            'P_a': P_a_A,
                            'P_b': P_b_B,
                            'P_ab': P_ab,
                            'P_ba': P_ba_swapped
                        })
                        break  # Cukup satu contoh per target
            if mixed_non_comm_details and mixed_non_comm_details[-1]['target'] == target:
                break  # Sudah dapat contoh untuk target ini
    
    if found_mixed_non_commutative:
        pair['mixed_non_comm_details'] = mixed_non_comm_details
        mixed_non_commutative_pairs.append(pair)

print(f"✓ Ditemukan {len(mixed_non_commutative_pairs)} pasangan dengan koefisien campuran tidak komutatif!")
print(f"  Dari total {len(commutative_pairs)} pasangan komutatif")

# Statistik per koefisien
mixed_stats = {7: 0, 5: 0}
for pair in mixed_non_commutative_pairs:
    if 'mixed_non_comm_details' in pair:
        for detail in pair['mixed_non_comm_details']:
            mixed_stats[detail['target']] += 1

print(f"\n  - Koefisien 7 (4+3): {mixed_stats[7]} pasangan")
print(f"  - Koefisien 5 (4+1, 3+2, dll): {mixed_stats[5]} pasangan")


ANALISIS TAMBAHAN: Koefisien Campuran (7 dan 5)

Cek: P_a(A) × P_b(B) vs P_b(B) × P_a(A) dimana a ≠ b
  - Koefisien 7: dari 4+3 atau 3+4
  - Koefisien 5: dari 4+1, 3+2, 2+3, atau 1+4
  FILTER: P_a(A) ≠ P_b(A) DAN P_a(B) ≠ P_b(B) DAN P_a(A) ≠ P_a(B) DAN P_b(A) ≠ P_b(B)
  FILTER CIRCULANT: Setidaknya salah satu dari A atau B bukan circulant

✓ Ditemukan 2384 pasangan dengan koefisien campuran tidak komutatif!
  Dari total 4752 pasangan komutatif

  - Koefisien 7 (4+3): 224 pasangan
  - Koefisien 5 (4+1, 3+2, dll): 2364 pasangan


In [104]:
# Verifikasi filter circulant
print("="*80)
print("VERIFIKASI FILTER CIRCULANT")
print("="*80)

# Gabungkan semua pasangan unik
all_pairs_keys = set()
for pair in non_commutative_perm_pairs:
    key = (pair['A_index'], pair['B_index'])
    all_pairs_keys.add(key)
for pair in mixed_non_commutative_pairs:
    key = (pair['A_index'], pair['B_index'])
    all_pairs_keys.add(key)

print(f"\nTotal pasangan unik: {len(all_pairs_keys)}")

# Cek berapa banyak yang kedua matriks circulant
both_circulant_count = 0
a_circulant_only = 0
b_circulant_only = 0
neither_circulant = 0

for key in all_pairs_keys:
    a_idx, b_idx = key
    A = all_latin_squares[a_idx]
    B = all_latin_squares[b_idx]
    
    a_is_circ = is_circulant(A)
    b_is_circ = is_circulant(B)
    
    if a_is_circ and b_is_circ:
        both_circulant_count += 1
    elif a_is_circ:
        a_circulant_only += 1
    elif b_is_circ:
        b_circulant_only += 1
    else:
        neither_circulant += 1

print(f"\nDistribusi circulant:")
print(f"  - Kedua A dan B circulant: {both_circulant_count} pasangan")
print(f"  - Hanya A circulant: {a_circulant_only} pasangan")
print(f"  - Hanya B circulant: {b_circulant_only} pasangan")
print(f"  - Tidak ada yang circulant: {neither_circulant} pasangan")

if both_circulant_count == 0:
    print(f"\n✓ SUKSES! Filter circulant bekerja dengan baik!")
    print(f"  Tidak ada pasangan dimana kedua matriks circulant.")
else:
    print(f"\n✗ WARNING! Masih ada {both_circulant_count} pasangan dengan kedua matriks circulant!")

# Tampilkan contoh pasangan dimana salah satu bukan circulant
print("\n" + "="*80)
print("CONTOH PASANGAN (setelah filter circulant)")
print("="*80)

shown = 0
for key in sorted(all_pairs_keys)[:3]:  # Ambil 3 contoh pertama
    a_idx, b_idx = key
    A = all_latin_squares[a_idx]
    B = all_latin_squares[b_idx]
    
    a_is_circ = is_circulant(A)
    b_is_circ = is_circulant(B)
    
    print(f"\nContoh #{shown + 1}:")
    print(f"Matriks A (circulant: {a_is_circ}):")
    print(A.astype(int))
    print(f"\nMatriks B (circulant: {b_is_circ}):")
    print(B.astype(int))
    print()
    
    shown += 1
    if shown >= 3:
        break

VERIFIKASI FILTER CIRCULANT

Total pasangan unik: 2556

Distribusi circulant:
  - Kedua A dan B circulant: 0 pasangan
  - Hanya A circulant: 0 pasangan
  - Hanya B circulant: 0 pasangan
  - Tidak ada yang circulant: 2556 pasangan

✓ SUKSES! Filter circulant bekerja dengan baik!
  Tidak ada pasangan dimana kedua matriks circulant.

CONTOH PASANGAN (setelah filter circulant)

Contoh #1:
Matriks A (circulant: False):
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B (circulant: False):
[[1 2 4 3]
 [2 3 1 4]
 [4 1 3 2]
 [3 4 2 1]]


Contoh #2:
Matriks A (circulant: False):
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B (circulant: False):
[[1 3 4 2]
 [3 2 1 4]
 [4 1 2 3]
 [2 4 3 1]]


Contoh #3:
Matriks A (circulant: False):
[[1 2 3 4]
 [2 1 4 3]
 [3 4 1 2]
 [4 3 2 1]]

Matriks B (circulant: False):
[[1 4 2 3]
 [4 3 1 2]
 [2 1 3 4]
 [3 2 4 1]]

